In [1]:
import os

In [2]:
# by default notebook runs in research folder
os.chdir("../../")
%pwd

'/Users/ashleyalexjacob/Data Science/Chicken-Disease-Classifer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.constants import *
from src.utils.commons import read_yaml,create_directories

In [24]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_url,
            unzip_dir= config.unzip_dir,
        )

        return data_ingestion_config
        

### Download Dataset

In [6]:
import os
import urllib.request as request
import zipfile
from src.logging import logger
from src.utils.commons import get_size

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url  = self.config.source_url,
                filename= self.config.local_data_file,
            )
            logger.info(f"{file_name} Downloaded Successfully ✅ with following info {headers}")
        
        else:
            print('IM in else')
            logger.info(f"File already exists with same size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Return: Nothing
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        os.remove(self.config.local_data_file)

### Pipeline

In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-13 00:16:00,341: INFO: commons: Yaml File: config/config.yaml Loaded]
[2023-06-13 00:16:00,353: INFO: commons: Yaml File: params.yaml Loaded]
[2023-06-13 00:16:00,359: INFO: commons: created directory at: artifacts]
[2023-06-13 00:16:00,363: INFO: commons: created directory at: artifacts/data_ingestion]
